#  Finding Similar Items: Textually Similar Documents

This notebook implements a pipeline to compute textual similarity between documents using:

- k-Shingles
- Jaccard similarity
- MinHash signatures
- Locality-Sensitive Hashing (LSH)

Dataset: BBC news articles (text files)
https://www.kaggle.com/datasets/shivamkushwaha/bbc-full-text-document-classification

Imports


In [68]:
import csv
import hashlib as h
import numpy as np
import os
import random
import numpy as np
import itertools


**Shingling Class**

**Purpose:**
The Shingling class is responsible for creating k-shingles from a document. Shingles can be either sequences of characters or words, depending on the boolean attribute word.

**Features:**

Removes special characters defined in removeCaracter to standardize text

Uses the md5 function from Python’s hashlib module to hash the shingles, which are required for the MinHashing step.

Supports both text (.txt) and CSV (.csv) files.

Provides methods to generate raw k-shingles, hashed shingles, and unique hashed shingles for further processing.

In [ ]:
class Shingling:
    k: int
    word: bool
    shingle: list[str]
    hashShingle: list[str]
    removeCaracter: list[str]

    def __init__(self, k: int = 1, word = False) -> None:
        self.k = k
        self.word = word
        self.removeCaracter = '()+=-_!,;.:/?'
    
    def _read_file(self, file_path: str) -> str:
        """
        Lecture du fichier .csv ou .txt
        Retourne tout le texte en minuscule.
        """
        ext = os.path.splitext(file_path)[1].lower()

        # Lecture CSV
        if ext == ".csv":
            with open(file_path, newline='', encoding='utf-8') as f:
                reader = csv.reader(f)
                text_list = []
                for row in reader:
                    if row and row[0].strip():
                        text_list.append(row[0].strip())
                return " ".join(text_list).lower()

        # Lecture TXT
        elif ext == ".txt":
            with open(file_path, "r", encoding="utf-8") as f:
                return f.read().lower()

        else:
            raise ValueError(f"Format de fichier non pris en charge : {file_path}")
        

    def kShingling(self, file_path: str) -> list[str]:
        text = self._read_file(file_path)
        kShingle = []

        #text = text.lower()
        
        for i in range(len(self.removeCaracter)):
            text = text.replace(self.removeCaracter[i],"")

        if(self.word):
            wordList = text.split(" ")
            for i in range(len(wordList)):
                if(i+self.k>len(wordList)):
                    self.shingle = kShingle
                    return kShingle
                key = ''
                for j in range(self.k):
                    key += wordList[i+j] + ' '
                key = key[:len(key)-1]
                kShingle.append(key)
        else:
            for i in range(len(text)):
                if(i+self.k>len(text)):
                    self.shingle = kShingle
                    return kShingle
                key = ''
                for j in range(self.k):
                    key += text[i+j]
                kShingle.append(key)
        self.shingle = kShingle
        return kShingle

    

    def hashShingling(self, file_path: str) -> list[str]:
        hashShingle = []
        kShingle = self.kShingling(file_path)
        for word in kShingle:
            hashShingle.append(h.md5(word.encode()))
        self.hashShingle = hashShingle
        return hashShingle
    
    def uniqueHashShingling(self, file_path: str) -> list[int]:
        hashShingle = []
        kShingle = self.kShingling(file_path)
        kShingle = np.unique(kShingle)
        for word in kShingle:
            hashShingle.append(h.md5(word.encode()).hexdigest())
        self.hashShingle = hashShingle
        return hashShingle

**CompareSets**
The CompareSets class computes the exact Jaccard similarity between two sets of hashed shingles.
Key Method:

getJaccardSim(set1, set2): Computes the ratio of the intersection over the union of two sets.

Use:
This provides the ground truth similarity for documents, which we can compare with the approximate similarity estimated by MinHashing.

In [70]:
class CompareSets:
    def __init__(self) -> None:
        print('Compare Sets class')
        
    def getJaccardSim(self,set1,set2) -> float:
        inter = 0
        union = 1
        for hash1 in set1:
            for hash2 in set2:
                if(hash1==hash2):
                    inter += 1
        union = len(np.unique(set1 + set2))
        return inter/union

**Minhash**

-Purpose:
The MinHashing class builds a MinHash signature for a document from its set of shingles. This signature is a vector of integers that approximates the Jaccard similarity between documents.

-Features:

Uses multiple hash functions of the form:
hi​(x)=(ai​⋅x+bi​)%c

a and b are random coefficients
c is a large prime.

-Result:
MinHash allows us to estimate similarity much faster than computing exact Jaccard, especially for large document sets.

Minhash version with permutations

In [71]:
# import numpy as np

# class MinHashing:
#     def __init__(self) -> None:
#         print('MinHashing class')
        
#     def buildSignature(self,set1,set2,nbPermut) -> float:
        
#         union = np.unique(set1 + set2)
#         mask1 = np.isin(union, set1)
#         mask2 = np.isin(union, set2)
        
#         signatureMatrix = np.empty((0, 2), int)
#         for _ in range(nbPermut):
#             permutedIndexes = np.random.permutation(len(union))
#             lineSignatureMatrix = np.zeros((1,2))
#             for i in range(len(permutedIndexes)):
#                 if(mask1[permutedIndexes[i]]):
#                     lineSignatureMatrix[0][0]=i
#                     break
#             for j in range(len(permutedIndexes)):
#                 if(mask2[permutedIndexes[j]]):
#                     lineSignatureMatrix[0][1]=j
#                     break
#             signatureMatrix = np.vstack((signatureMatrix, lineSignatureMatrix))

#         return signatureMatrix

MinHashing (Using Hashing functions)

In [72]:
import numpy as np
import random

class MinHashing:
    def __init__(self, num_hashes=100, max_shingle=2**32 - 1, seed=42):
        self.num_hashes = num_hashes
        self.max_shingle = max_shingle
        random.seed(seed)

        # Calculating the coeffs for the hashing fct: a and b, h_i(x) = (a_i * x + b_i) % c
        self.a = [random.randint(1, max_shingle - 1) for _ in range(num_hashes)]
        self.b = [random.randint(0, max_shingle - 1) for _ in range(num_hashes)]
        self.c = 4294967311 # prime number + must be > max(shingle) , shingles are converted into integers after the MD5

    def compute_signature(self, shingle_set):
        """
        Signature for one doc returns a vector of integers
        """
        sig = np.full(self.num_hashes, np.inf)

        for shingle in shingle_set:
            for i in range(self.num_hashes):
                h_val = (self.a[i] * shingle + self.b[i]) % self.c
                if h_val < sig[i]:
                    sig[i] = h_val

        return sig.astype(int)


**CompareSignatures**

Purpose:
The CompareSignatures class computes the similarity between two MinHash signatures.

Key Method:

computeEstimateSimilarity(signature_matrix): Returns the fraction of positions where the two signatures are identical.

Use:
This gives an approximate similarity between documents without using the full sets of shingles.

In [73]:
class CompareSignatures:
    def __init__(self) -> None:
        print('Compare Signatures class')
        
    def computeEstimateSimilarity(self,signature) -> float:
        
        signature = signature.T

        y = signature[0]-signature[1]
        y = np.where(y==0,1,0)
        
        return np.sum(y)/len(y)

**LocalSensitiveHash**

Purpose:

The LocalSensitiveHash class implements the Locality Sensitive Hashing (LSH) technique to efficiently find candidate pairs of similar documents. Instead of comparing every pair of documents (which can be very slow for large datasets), LSH reduces the number of comparisons by focusing only on likely similar pairs.

Steps:

Banding:

The MinHash signature of each document is divided into band bands.

Each band contains r consecutive rows (elements) from the signature.

This splits the signature matrix into smaller pieces that can be hashed separately.

Candidate Detection:

For a given pair of documents, the method compares each corresponding band.

If the hash of a band matches for the two documents, that band contributes to their similarity score.

A pair is considered a candidate if the fraction of matching bands is above the threshold.

Hashing Bands:

Each band is converted to a string and hashed using md5.

Matching hashes indicate that the band is identical for both documents.

In [ ]:
# class LocalSensitiveHash:
#     band: int
#     r: int
#     threshold: int

#     def __init__(self, band : int, threshold : int) -> None:
#         self.band = band
#         self.threshold = threshold

#     def lookForCandidates(self,signature,col1 : int = 0,col2 : int = 1) -> float:
        
#         self.r = int(len(signature)/self.band)
#         similarities = 0
#         if(self.r * self.band == len(signature)):
#             signature = signature.T #transpose the signature matrix so that each column corresponds to a document.
#             for i in range(self.band):
#                 band1 = str(signature[col1][i:i+self.r])
#                 band2 = str(signature[col2][i:i+self.r])
                
#                 if(h.md5(band1.encode()).hexdigest()==h.md5(band2.encode()).hexdigest()):
#                     similarities += 1
#                     print("similarity found")
#             print(f"similarities = {similarities/self.band}")
#             if(similarities/self.band >= self.threshold):
#                 return 1
#             else:
#                 return 0
            
#         else:
#             print("Wrong size")
#             return -1
import hashlib as h
import numpy as np
from collections import defaultdict
from itertools import combinations

class LocalSensitiveHashMulti:
    """
    LSH for many documents.
    """
    def __init__(self, band: int = 20, threshold: float = 0.7):
        self.band = band
        self.threshold = threshold
        self.r = None  # rows per band

    def find_candidates(self, signatures: dict):
        """
        signatures: dict {doc_name: signature_vector }
        Returns a set of candidate pairs (doc1, doc2)
        """
        # Convert signatures dict to matrix form for convenience
        doc_names = list(signatures.keys())
        sig_matrix = np.array([signatures[doc] for doc in doc_names])
        sig_matrix = sig_matrix.T  # rows = hash values
                                    #columns = documents

        self.r = sig_matrix.shape[0] // self.band
        if self.r * self.band != sig_matrix.shape[0]:
            raise ValueError("Number of hash functions must be divisible by number of bands")

        candidates = set()

        for b in range(self.band):
            band_hash_table = defaultdict(list)
            start = b * self.r
            end = (b + 1) * self.r

            # Hash each document's band
            for col, doc in enumerate(doc_names):
                band = sig_matrix[start:end, col]
                band_str = ",".join(map(str, band))
                band_hash = h.md5(band_str.encode()).hexdigest()
                band_hash_table[band_hash].append(doc)

            # Any documents sharing the same band hash are candidate pairs
            for bucket_docs in band_hash_table.values():
                if len(bucket_docs) > 1:
                    for doc1, doc2 in combinations(bucket_docs, 2):
                        candidates.add(tuple(sorted([doc1, doc2])))

        return candidates


 First test with 2 simple sentences

In [75]:
# LSHasher = LocalSensitiveHash(25,0.8) 
# estimator = CompareSignatures()
# miniHasher = MinHashing()
# comparator = CompareSets()

# fiveShingler = Shingling(3,False)

# #kShingle = fiveShingler.kShingling('test.csv')
# #hashList = fiveShingler.uniqueHashShingling('test.csv')
# #print(f"len = {len(hashList)}")
# hashShingle = fiveShingler.uniqueHashShingling('1.csv')
# test3 = fiveShingler.uniqueHashShingling('2.csv')
# #print(kShingle)
# #print(hashShingle)

# similarity = comparator.getJaccardSim(hashShingle,test3)
# signature = miniHasher.buildSignature(hashShingle,test3,nbPermut=10000)
# estimate = estimator.computeEstimateSimilarity(signature)
# #print(f"longueur de la signature {len(signature)}")
# retour = LSHasher.lookForCandidates(signature)
# print(f"r = {LSHasher.r}, retour de LSH {retour}")

# #print(signature)
# print(similarity)
# print(estimate)

# ------------------- TEST EXAMPLE -------------------
In this test, we create two very similar short documents.Both sentences describe "the quick brown fox jumping over the lazy dog",with the second document containing just one additional phrase ("et joue").This allows us to verify whether our pipeline (Shingling → MinHash → LSH)correctly detects similarity between documents that only differ slightly

In [76]:
docs = ["doc1.txt", "doc2.txt"]
texts = [
    "le rapide renard brun saute par dessus le chien paresseux",
    "le rapide renard brun saute par dessus le chien paresseux et joue"
]
for name, txt in zip(docs, texts):
    with open(name, "w", encoding="utf-8") as f:
        f.write(txt)

# ------------------- Classes -------------------
Shinglers = Shingling(k=3, word=False)
comparator = CompareSets()
miniHasher = MinHashing(num_hashes=100)
estimator = CompareSignatures()
LSHasher = LocalSensitiveHashMulti(band=25, threshold=0.6)

# ------------------- Shingles and signatures -------------------
shingles_dict = {}
signatures = {}
for doc in docs:
    shingles = Shinglers.uniqueHashShingling(doc)
    shingles_int = [int(x,16) for x in shingles]
    shingles_dict[doc] = shingles_int
    signatures[doc] = miniHasher.compute_signature(shingles_int)

# ------------------- Comparing -------------------
sig_matrix = np.array([signatures[docs[0]], signatures[docs[1]]]).T
candidate = LSHasher.find_candidates(signatures)
exact = comparator.getJaccardSim(shingles_dict[docs[0]], shingles_dict[docs[1]])
estimate = estimator.computeEstimateSimilarity(sig_matrix)

print(f"\nComparing {docs[0]} <-> {docs[1]}")
print(f"Candidate (LSH): {candidate}")
print(f"Jaccard exact: {exact:.4f}")
print(f"Estimated MinHash: {estimate:.4f}")

Compare Sets class
Compare Signatures class

Comparing doc1.txt <-> doc2.txt
Candidate (LSH): {('doc1.txt', 'doc2.txt')}
Jaccard exact: 0.8621
Estimated MinHash: 0.8600


# -------------Testing with many docs ----------------

**Loading Dataset** (Business)

In [77]:
# --- Files to compare ---
#docs = ["1.csv", "2.csv", "3.csv","4.csv","5.csv"]
folder_path = "data/business"  

# --- Extracting all files ---
docs = [
    os.path.join(folder_path, f)
    for f in os.listdir(folder_path)
    if f.endswith(".txt") or f.endswith(".csv")
]

In [78]:
# ------------------- Classes -------------------
Shinglers = Shingling(k=3, word=False)
comparator = CompareSets()
miniHasher = MinHashing(num_hashes=100)
estimator = CompareSignatures()
LSHasher = LocalSensitiveHashMulti(band=20, threshold=0.7)

# ------------------- Shingles and signatures -------------------
shingles_dict = {}
signatures = {}
for doc in docs:
    shingles = Shinglers.uniqueHashShingling(doc)
    shingles_int = [int(x,16) for x in shingles]
    shingles_dict[doc] = shingles_int
    signatures[doc] = miniHasher.compute_signature(shingles_int)

# ------------------- Comparing -------------------
import itertools
import numpy as np
# Open a CSV file to write the results
# --- Find all candidate pairs using LSHMulti ---
candidates = LSHasher.find_candidates(signatures)  # signatures = dict {doc_name: signature}

# Open CSV to write results
with open("lsh_results.csv", mode="w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Document 1", "Document 2", "LSH Candidate", "Jaccard Exact", "MinHash Estimate"])

    for doc1, doc2 in itertools.combinations(docs, 2):
        sig_matrix = np.array([signatures[doc1], signatures[doc2]]).T

        # Check if this pair is an LSH candidate
        is_candidate = 1 if (doc1, doc2) in candidates else 0
        if is_candidate:
            print(f"LSH candidate pair: {doc1} <-> {doc2}")

        # Exact Jaccard
        exact = comparator.getJaccardSim(shingles_dict[doc1], shingles_dict[doc2])

        # Estimated MinHash similarity
        estimate = estimator.computeEstimateSimilarity(sig_matrix)

        # Write row
        writer.writerow([doc1, doc2, is_candidate, f"{exact:.4f}", f"{estimate:.4f}"])

# with open("lsh_results.csv", mode="w", newline="", encoding="utf-8") as f:
#     writer = csv.writer(f)
    
#     # Write the header
#     writer.writerow(["Document 1", "Document 2", "LSH Candidate", "Jaccard Exact", "MinHash Estimate"])
#     for doc1, doc2 in itertools.combinations(docs, 2):
#         sig_matrix = np.array([signatures[doc1], signatures[doc2]]).T
        
#         # LSH
#         candidate = LSHasher.find_candidates(signatures)
#             # printing directly the matching candidate
#         if candidate == 1:
#             print(f"LSH candidate pair: {doc1} <-> {doc2}")
        
#         # Jaccard exact
#         exact = comparator.getJaccardSim(shingles_dict[doc1], shingles_dict[doc2])
        
#         # Estimation MinHash
#         estimate = estimator.computeEstimateSimilarity(sig_matrix)
#                 # Write the row
#         writer.writerow([doc1, doc2, candidate, f"{exact:.4f}", f"{estimate:.4f}"])
    
#     print(f"\nComparing {doc1} <-> {doc2}")
#     print(f"Candidate (LSH): {candidate}")
#     print(f"Jaccard exact: {exact:.4f}")
#     print(f"Estimated MinHash: {estimate:.4f}")


Compare Sets class
Compare Signatures class
LSH candidate pair: data/business\001.txt <-> data/business\015.txt
LSH candidate pair: data/business\002.txt <-> data/business\031.txt
LSH candidate pair: data/business\002.txt <-> data/business\053.txt
LSH candidate pair: data/business\002.txt <-> data/business\087.txt
LSH candidate pair: data/business\003.txt <-> data/business\060.txt
LSH candidate pair: data/business\004.txt <-> data/business\027.txt
LSH candidate pair: data/business\004.txt <-> data/business\030.txt
LSH candidate pair: data/business\004.txt <-> data/business\077.txt
LSH candidate pair: data/business\004.txt <-> data/business\090.txt
LSH candidate pair: data/business\005.txt <-> data/business\047.txt
LSH candidate pair: data/business\005.txt <-> data/business\067.txt
LSH candidate pair: data/business\005.txt <-> data/business\082.txt
LSH candidate pair: data/business\006.txt <-> data/business\049.txt
LSH candidate pair: data/business\007.txt <-> data/business\019.txt
LSH 

**Some plots/comparing results**

# -----------Visualising Jaccard Similarity-----------

In this section, we visualize the exact Jaccard similarities between all document pairs.
This helps us understand the overall similarity structure within the dataset
and validate whether similar documents are indeed detected as such.
For the dataset, we used 5 mails having some similarities

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# import numpy as np

# N = len(docs)
# jaccard_matrix = np.zeros((N, N))
# Compute pairwise Jaccard similarity between all document pairs
# for i, doc1 in enumerate(docs):
#     for j, doc2 in enumerate(docs):
#         jaccard_matrix[i, j] = comparator.getJaccardSim(shingles_dict[doc1], shingles_dict[doc2])
# We use Seaborn to create a heatmap showing similarity values between all documents.
# The heatmap provides a quick visual overview of which documents are most similar.
# sns.heatmap(jaccard_matrix, annot=True, xticklabels=docs, yticklabels=docs, cmap="viridis")
# plt.title("Exact Jaccard Similarity Matrix")
# plt.show()


#  VISUALIZING MINHASH ESTIMATED SIMILARITIES 
This section computes and visualizes the *estimated* document similarities obtained using the MinHash signatures (instead of the exact Jaccard similarity).

In [ ]:
# minhash_matrix = np.zeros((N, N))

# for i, doc1 in enumerate(docs):
#     for j, doc2 in enumerate(docs):
#         # matrice des signatures pour le computeEstimateSimilarity
#         sig_matrix = np.array([signatures[doc1], signatures[doc2]]).T
#         minhash_matrix[i, j] = estimator.computeEstimateSimilarity(sig_matrix)

# # --- Heatmap ---
# Similar to the previous visualization, we now display the estimated similarities
# in the form of a heatmap using Seaborn.
# sns.heatmap(minhash_matrix, annot=True, xticklabels=docs, yticklabels=docs, cmap="viridis")
# plt.title("Estimated MinHash Similarity Matrix")
# plt.show()